In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import torch
import random
import torch.nn as nn
from torch import optim
from torch.autograd import variable
import torch.nn.functional as F
import sys

In [2]:
use_cuda = True
TRAIN = False

In [3]:
for arg in sys.argv:
    if arg =='--train':
        TRAIN = True
    elif arg =='--cuda':
        use_cuda = torch.cuda.is_available()

In [5]:
print("CUDA :", use_cuda)
print("TRAIN: ", TRAIN)

CUDA : True
TRAIN:  False


**Indexing words**: making helper class lang that has word to index and index to word mappings.

In [6]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__ (self,name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2 #count of SOS and EOS
        
    def add_sentence(self,sentence):
        for word in sentence.split(' '):
            self.add_word(word)
            
    def add_word(self,word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words +=1
        else:
            self.word2count[word] +=1
            

        

**Reading and decoding files from Unicode to ascii**

In [7]:
def unicode2ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD',s)
        if unicodedata.category(c) != 'Mn'
    )

**Lowercase, trim and remove non letter characters**